# Imports

In [133]:
import pandas as pd
import plotly.express as px
import math

In [134]:
users = pd.read_csv("datas/users.csv")

In [135]:
accidents_by_sex = users.sexe.value_counts() / users.shape[0]

px.bar(
    x=['Homme', 'Femme'],
    y=accidents_by_sex,
    text=(accidents_by_sex * 100).apply(lambda x: str(math.floor(x)) + '%'),
    range_y=[0, 1],
    labels={'x': 'Sexe', 'y': 'Pourcentage'}
)